In [59]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
prompt_template = ChatPromptTemplate([
    ('system','''
    이곳이 처음이거나 자주 오는 이들에게도 모르는 것이 있다면 언제든 대답해주는 게 네 역할이지.



마사회
You are a Virtual Assistant AI designed to serve as an information desk guide for the Korea Racing Authority (KRA). Your primary role is to provide clear, friendly, and detailed answers to visitors’ questions. Follow these steps when assisting users:

Step 1: Document Analysis

Review the uploaded documents or reference materials provided to identify relevant information regarding the user’s question.

Step 2: Database Search or Supplementary Research

If the documents lack sufficient details, search through available internal databases or reference reliable sources to gather the required information.

Step 3: Answer Formulation

Combine the extracted data to craft a concise, accurate, and visitor-friendly response. Tailor your tone to be professional yet approachable.

Step 4: Simplification and Clarity

Ensure the information is easy to understand, avoiding technical jargon unless specifically requested by the user.

Step 5: Summary and Organization

Provide a brief summary of your answer and, where applicable, suggest any additional services or information that might enhance the visitor’s experience.

Now, proceed to execute this task: “Provide answers to visitor inquiries based on available documents and additional resources.” Take a step-by-step approach to ensure the response is comprehensive and accurate.''' ),
    
    ('user', '{question}')
])

# 출력 파서 생성
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

## pdf 파일 로드

In [60]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_community.document_loaders import PyPDFLoader
import os

# 여러 PDF 파일 경로를 리스트로 생성
pdf_folder = 'C:/Workspaces/project_ma/pdfs'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# 각 PDF 파일을 로드하여 문서로 처리
pdf_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents.extend(loader.load())  # 문서를 리스트에 추가

print(f"Loaded {len(pdf_documents)} documents.")

Loaded 61 documents.


In [62]:
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [document.page_content for document in pdf_documents]

all_documents = pdf_documents #+ web_documents

In [63]:
len(all_documents)

61

In [53]:
all_documents[30:35]

[Document(metadata={'source': 'C:/Workspaces/project_ma/pdfs\\20241221_SE_all_races.pdf', 'page': 10}, page_content='4 2024-12-21T13:10:00+09:00\n4 2024-12-21T13:10:00+09:00\n4 2024-12-21T13:10:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n5 2024-12-21T14:00:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n6 2024-12-21T14:50:00+09:00\n7 2024-12-21T15:40:00+09:00\n7 2024-12-21T15:40:00+09:00\n7 2024-12-21T15:40:00+09:00\n7 2024-12-21T15:40:00+09:00\n7 2024-12-21T15:40:00+09:00\n7 2024-12-21T

In [39]:
for doc in all_documents[19:21]:
    print(doc.page_content)

8 2024-12-20T18:00:00+09:00
8 2024-12-20T18:00:00+09:00
8 2024-12-20T18:00:00+09:00
8 2024-12-20T18:00:00+09:00
items.item.age 나이
items.item.chulNo 출전 번호
items.item.hrName 말 이름
items.item.hrOrd1CntT 통산 1등횟수
items.item.hrOrd2CntT 통산 2등횟수
items.item.hrRcCntT 통산 출주횟수
items.item.jkName 기수 이름
items.item.meet 출전 경기장
items.item.rank 출전 게임 랭크
items.item.rcDate 경기 날짜
items.item.rcNo 경기 번호
items.item.schStTime 경기 시작 시간


## Vector Store 구축 - FAISS 사용

In [64]:
from langchain.vectorstores import FAISS

vectors = FAISS.from_documents(all_documents, embeddings) #documents : Document
vectors

In [65]:
results = vectors.similarity_search("부산", k=3)
print(len(results))
# for i, doc in enumerate(results):
#     print(i, doc.page_content)

print(results[2])

3
page_content='items.item.age items.item.chulNo items.item.hrName
3 1 한산
2 10 장타호령
3 2 갈채
2 3 스타덤
3 4 파도소리
3 5 평화오름
3 6 출사표
2 7 환영파티
3 8 킹앤탑
2 9 신통방통
3 1 월등한
3 10 백호일마
2 2 원더풀대양
3 3 핸섬보이
2 4 마패도끼
2 5 대군주
3 6 청신호
2 7 출품작
3 8 자유시대
3 9 황해명장
3 1 명품챔프
4 10 청마여제
3 2 애조로
3 3 가온빛
3 4 신조
5 5 봉성여걸
3 6 캐논공주
4 7 붉은등불
3 8 금빛여제
4 9 걸작탄생
4 1 명품함성
2 10 등고자비
2 2 캡틴공주
3 3 질풍태양
2 4 한라봉
2 5 별빛찬란
4 6 드림더킹
3 7 서부두
3 8 혜성스타
2 9 정정당당
4 1 웅진퍼스트
6 10 선덕공주' metadata={'source': 'C:/Workspaces/project_ma/pdfs\\20241220_JJ_all_races.pdf', 'page': 0}


## Retriever 객체 생성

In [66]:
retriever = vectors.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024EC1B25670>, search_kwargs={})

In [71]:
# 벡터스토어 검색 및 LLM 질의

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
)

#질의 객체
rtvr_qa=RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type = 'stuff' #검색된 내용을 하나의 텍스트로 생성해서 LLM에 전달
)

#llm 질의
response1 = rtvr_qa.invoke('22일 부경 경기장의 경기들에 대해 알려줘')
print(response1)

{'query': '22일 부경 경기장의 경기들에 대해 알려줘', 'result': '2024년 12월 22일에 부경 경기장에서 열리는 경기는 다음과 같습니다:\n\n- 국6등급 경기:\n  - 기수: 최은경, 정동철, 김혜선, 이효식, 진겸, 김태현, 최시대, 정도윤, 먼로, 다나카, 우에다, 김어수, 신윤섭, 서승운, 박재이, 송경윤, 전진구\n\n- 국5등급 경기:\n  - 기수: 권오찬, 최은경, 김동영, 김혜선, 최시대, 정도윤, 먼로, 서승운, 전진구, 이성재, 유광희, 진겸\n\n- 혼4등급 경기:\n  - 기수: 김어수, 먼로, 김동영, 김혜선, 우에다, 최은경, 최시대, 이성재, 송경윤, 김태현, 서승운, 전진구\n\n- 국3등급 경기:\n  - 기수: 김혜선, 먼로, 최은경, 박재이, 최시대, 김어수, 정동철, 이성재, 김동영, 우에다, 정도윤, 이효식\n\n- 2등급 경기:\n  - 기수: 먼로, 송경윤, 서승운, 다나카, 박재이, 전진구, 이효식, 정도윤, 진겸, 김혜선, 유광희, 최은경\n\n이 경기는 모두 부경 경기장에서 열리며, 각 등급별로 다양한 기수들이 참여합니다.'}


In [29]:
#response1 = rtvr_qa.invoke('오토벳에 대해 알려줘')
#print(response1)
#TODO pdf 업로드 시, 내용이 누락될 가능성?
#race_guide.pdf에 오토벳에 대한 내용이 있는데 뱉어주는 게 없다.
#민하님께서 guide.pdf만 넣고 돌렸을 땐 관련 내용이 잘 나왔는데, 여러 pdf 파일을 넣고 돌리니깐 해당 내용을 제공할 수 없단 답변을 받았다고 하심.
#이에 다 같이 돌려봤으나 똑같은 증상을 겪었다.